In [1]:
import os
# Change the terminal placement to the folder where the py files  are located  
os.chdir('/Users/ilyeshammouda/Desktop/Ilyes/2A ENSAE/stage/code_stage_2_A_ENSAE/Stage_2A_ENSAE/code') 
# We position ourselves in the directory where the main classes are located.
print(f" You are currently in the directory: {os.getcwd()}")

 You are currently in the directory: /Users/ilyeshammouda/Desktop/Ilyes/2A ENSAE/stage/code_stage_2_A_ENSAE/Stage_2A_ENSAE/code


In [2]:
from Zoro import *
from functions import *
from Zoro_MD import * 
from help_function import *

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy

In [3]:
mpl.style.use('seaborn')

In [4]:
# problem set up
n = 2000
s = int(0.1*n)
# noise amplitude
noiseamp = 0.001
#noiseamp = 0 
decay_factor=0.5
x_star=tools.sparse_vector(n=n,s=s)
#obj_func = SparseQuadric(n, s, noiseamp)
#obj_func=CompressibleQuadric(n=n, decay_factor=decay_factor, noiseamp=noiseamp)
#obj_func=SparseQuadric(n, s, noiseamp)
obj_func=weakly_sparse(n,x_star, noiseamp)
# Choose initialization
x0    = np.random.randn(n)
x0    = 100*x0/np.linalg.norm(x0)
xx0   = copy.deepcopy(x0)

sparsity = s
#sparsity = int(0.1*len(x0)) # This is a decent default, if no better estimate is known. 

# Parameters for ZORO. Defaults are fine in most cases
params = {"step_size":0.5, "delta": 0.0001, "max_cosamp_iter": 10, 
          "cosamp_tol": 0.5,"sparsity": sparsity,
          "num_samples": int(np.ceil(np.log(len(x0))*sparsity))}

performance_log_ZORO = [[0, obj_func(x0)]]

In [5]:
#IHT set up 
lamda_IHT=10 # lambda initialisation 
threshold_IHT=0.01 # lambda threshold
C_IHT=0.9 # The update param for the lambda_k at each itteration.  
step_IHT=1/(3*n) # The step of the IHT algo 
itt_IHT=40  # The maximum number of itterations for the IHT algo



In [6]:
#ISTA set up 
lamda_ISTA=10 # lambda initialisation 
threshold_ISTA=0.01 # lambda threshold
C_ISTA=0.9 # The update param for the lambda_k at each itteration.  
step_ISTA=1/(3*n) # The step of the IHT algo 
itt_ISTA=40  # The maximum number of itterations for the IHT algo



In [7]:
#Original Version of the Zoro Algo with different algos for estimating the gradient
opt_Co  = ZORO(x0, obj_func, params,algo='CoSaMP', function_budget= int(1e6),x_star=x_star)
opt_Lasso  = ZORO(x0, obj_func, params,algo='Lasso', function_budget= int(1e6),itt_Lasso=30)
opt_DLasso  = ZORO(x0, obj_func, params,algo='DLasso', function_budget= int(1e6))
opt_IHT  = ZORO(x0, obj_func, params,algo='IHT_Classique', function_budget= int(1e6),s=sparsity,itt_IHT=itt_IHT)
opt_IHT_ad  = ZORO(x0, obj_func, params,algo='IHT_ad', function_budget= int(1e6),threshold_IHT=threshold_IHT,C_IHT=C_IHT,lamda_IHT=lamda_IHT,itt_IHT=itt_IHT,step_IHT=step_IHT)
#opt_ista_ad  = ZORO(x0, obj_func, params,algo='ISTA_ad', function_budget= int(1e6),threshold_IHT=threshold_ista,C_IHT=C_ista,lamda_IHT=lamda_ista,itt_IHT=itt_ista,step_IHT=step_ista)
# the actual optimization routine
termination=False

In [8]:
#Original the Zoro Algo using MD and  with different algos for estimating the gradient
opt_IHT_MD  = ZORO_MD(x0, obj_func, params,algo='IHT_Classique', function_budget= int(1e6),s=sparsity,itt_IHT=itt_IHT,x_star=x_star)
opt_Co_MD  = ZORO_MD(x0, obj_func, params,algo='CoSaMP', function_budget= int(1e6),s=sparsity,itt_IHT=itt_IHT,x_star=x_star)
opt_IHT_ad_MD  = ZORO_MD(x0, obj_func, params,algo='IHT_ad', function_budget= int(1e6),s=sparsity,itt_IHT=itt_IHT,x_star=x_star)
opt_Lass_MD  = ZORO_MD(x0, obj_func, params,algo='Lasso', function_budget= int(1e6),itt_Lasso=100,tol_Lasso=0.01)
opt_DLasso_MD  = ZORO_MD(x0, obj_func, params,algo='DLasso', function_budget= int(1e6),itt_Lasso=100,tol_Lasso=0.01)

# Perform the original Zoro algorithm with different methods for estimating the gradient

In [ ]:
performance_log_ZORO_Co,cost_Co=opt_Co.Zoro()
performance_log_ZORO_IHT,cost_IHT=opt_IHT.Zoro()
#performance_log_ZORO_Lasso,cost_Lasso=opt_Lasso.Zoro()
#performance_log_ZORO_DLasso,cost_DLasso=opt_DLasso.Zoro()
performance_log_ZORO_IHT_ad,cost_IHT_ad=opt_IHT_ad.Zoro()


# Perform the Zoro algorithm with MD and different methods for estimating the gradient

In [9]:
performance_log_ZORO_Co_MD,cost_Co_MD=opt_Co_MD.Zoro_MD()

Estimated f(x_k): 10110.034335 x_k-x_star: 26.511291 function evals: 3042
Estimated f(x_k): 702.848541 x_k-x_star: 10.385804 function evals: 6084
Estimated f(x_k): 107.864940 x_k-x_star: 8.301681 function evals: 9126
Estimated f(x_k): 68.917874 x_k-x_star: 8.126710 function evals: 12168
Estimated f(x_k): 66.043438 x_k-x_star: 8.108859 function evals: 15210
Estimated f(x_k): 65.753606 x_k-x_star: 8.106038 function evals: 18252
Estimated f(x_k): 65.707860 x_k-x_star: 8.105416 function evals: 21294
Estimated f(x_k): 65.697707 x_k-x_star: 8.105242 function evals: 24336
Estimated f(x_k): 65.694944 x_k-x_star: 8.105176 function evals: 27378
Estimated f(x_k): 65.693902 x_k-x_star: 8.105139 function evals: 30420
Estimated f(x_k): 65.693323 x_k-x_star: 8.105103 function evals: 33462
Estimated f(x_k): 65.692692 x_k-x_star: 8.105066 function evals: 36504
Estimated f(x_k): 65.692096 x_k-x_star: 8.105030 function evals: 39546
Estimated f(x_k): 65.691474 x_k-x_star: 8.104993 function evals: 42588
Es

In [ ]:
performance_log_ZORO_IHT_MD,cost_IHT_MD=opt_IHT_MD.Zoro_MD()

In [ ]:
performance_log_ZORO_Lasso_MD,cost_Lasso_MD=opt_Lass_MD.Zoro_MD()

In [ ]:
performance_log_ZORO_IHT_ad_MD,cost__IHT_ad_MD=opt_IHT_ad_MD.Zoro_MD()


In [ ]:
performance_log_ZORO_DLasso_MD,cost_DLasso_MD=opt_DLasso_MD.Zoro_MD()

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(performance_log_ZORO_Lasso_MD)[:,0],
 (np.log10(np.array(performance_log_ZORO_Lasso_MD)[:,1])), linewidth=1, label = "ZORO Lasso MD")
#ax.plot(np.array(performance_log_ZORO_Co_MD)[:,0],
 #(np.array(performance_log_ZORO_Co_MD)[:,1]), linewidth=1, label = "ZORO_MD")
ax.plot(np.array(performance_log_ZORO_IHT_MD)[:,0],
 (np.log10(np.array(performance_log_ZORO_IHT_MD)[:,1])), linewidth=1, label = "ZORO IHT  MD")
ax.plot(np.array(performance_log_ZORO_Co_MD)[:,0],
 (np.log10(np.array(performance_log_ZORO_Co_MD)[:,1])), linewidth=1, label = "ZORO original MD")
ax.plot(np.array(performance_log_ZORO_IHT_ad_MD)[:,0],
 (np.log10(np.array(performance_log_ZORO_IHT_ad_MD)[:,1])), linewidth=1, label = "ZORO IHT ad MD")
ax.plot(np.array(performance_log_ZORO_DLasso_MD)[:,0],
 (np.log10(np.array(performance_log_ZORO_DLasso_MD)[:,1])), linewidth=1, label = "ZORO DLasso MD")
ax.plot(np.array(performance_log_ZORO_Co)[:,0],
 (np.log10(np.array(performance_log_ZORO_Co)[:,1])), linewidth=1, label = "ZORO Original")
ax.plot(np.array(performance_log_ZORO_IHT)[:,0],
 (np.log10(np.array(performance_log_ZORO_IHT)[:,1])), linewidth=1, label = "ZORO Original IHT ")
ax.plot(np.array(performance_log_ZORO_IHT_ad)[:,0],
 (np.log10(np.array(performance_log_ZORO_IHT_ad)[:,1])), linewidth=1, label = "ZORO Original IHT ad ")

plt.xlabel('number of queries')
plt.ylabel('$log_{10}(f(x_{k})-f^{*})$')
leg = ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.array(cost_Co)[:,0],
 (np.array(cost_Co)[:,1]), linewidth=1, label = "ZORO original")

ax.plot(np.array(cost_IHT)[:,0],
 (np.array(cost_IHT)[:,1]), linewidth=1, label = "ZORO original IHT")
ax.plot(np.array(cost_IHT_ad)[:,0],
 (np.array(cost_IHT_ad)[:,1]), linewidth=1, label = "ZORO original IHT ad ")
ax.plot(np.array(cost_Co_MD)[:,0],
 (np.array(cost_Co_MD)[:,1]), linewidth=1, label = "ZORO original MD")
ax.plot(np.array(cost_IHT_MD)[:,0],
 (np.array(cost_IHT_MD)[:,1]), linewidth=1, label = "ZORO IHT MD")
ax.plot(np.array(cost__IHT_ad_MD)[:,0],
 (np.array(cost__IHT_ad_MD)[:,1]), linewidth=1, label = "ZORO IHT ad MD")
ax.plot(np.array(cost_Lasso_MD)[:,0],
 (np.array(cost_Lasso_MD)[:,1]), linewidth=1, label = "ZORO Lasso MD")
ax.plot(np.array(cost_DLasso_MD)[:,0],
 (np.array(cost_DLasso_MD)[:,1]), linewidth=1, label = "ZORO DLasso MD")

plt.xlabel('number of queries')
plt.ylabel('$ || x_{k}-x{*}||_{2} $')
leg = ax.legend()
plt.show()